In [54]:
import pandas as pd
import numpy as np
import seaborn as sns
from numba import jit


In [55]:
train=pd.read_csv('train.csv')
test=pd.read_csv("test.csv")
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [56]:
(train[train["Cabin"].isnull()==False]["Survived"]==1).mean()

0.66666666666666663

In [57]:
(test[test["Cabin"].isnull()==True]).describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,327.000000,327.000000,245.000000,327.000000,327.000000,326.000000
mean,1100.088685,2.571865,27.085714,0.412844,0.373089,19.080828
std,119.523394,0.641273,12.339533,0.954913,1.015951,24.876658
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,998.500000,2.000000,21.000000,0.000000,0.000000,7.822900
50%,1102.000000,3.000000,25.000000,0.000000,0.000000,11.500000
75%,1201.500000,3.000000,33.000000,1.000000,0.000000,22.274975
max,1309.000000,3.000000,63.000000,8.000000,9.000000,262.375000


In [58]:
train["Cabin"].isnull().sum().sum()

687

In [59]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [60]:
342/891

0.3838383838383838

In [61]:
train["Survived"].mean()

0.3838383838383838

In [62]:
train['Age'].mean()

29.69911764705882

In [63]:
train['Age'].astype('str')

0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
5       nan
6      54.0
7       2.0
8      27.0
9      14.0
10      4.0
11     58.0
12     20.0
13     39.0
14     14.0
15     55.0
16      2.0
17      nan
18     31.0
19      nan
20     35.0
21     34.0
22     15.0
23     28.0
24      8.0
25     38.0
26      nan
27     19.0
28      nan
29      nan
       ... 
861    21.0
862    48.0
863     nan
864    24.0
865    42.0
866    27.0
867    31.0
868     nan
869     4.0
870    26.0
871    47.0
872    33.0
873    47.0
874    28.0
875    15.0
876    20.0
877    19.0
878     nan
879    56.0
880    25.0
881    33.0
882    22.0
883    28.0
884    25.0
885    39.0
886    27.0
887    19.0
888     nan
889    26.0
890    32.0
Name: Age, Length: 891, dtype: object

In [64]:
train["Age"] = train["Age"].replace(to_replace=[None] ,value=[29])
test["Age"] = test["Age"].replace(to_replace=[None] ,value=[29])
train["Embarked"] = train["Embarked"].replace(to_replace=[None] ,value=['S'])
test["Embarked"] = test["Embarked"].replace(to_replace=[None] ,value=['S'])
#train["Cabin"] = train["Cabin"].replace(to_replace=[None],value=[0])
#test["Cabin"] = test["Cabin"].replace(to_replace=[None],value=[0])

In [65]:
train['Age'].astype("int64")
test['Age'].astype("int64")

0      34
1      47
2      62
3      27
4      22
5      14
6      30
7      26
8      18
9      21
10     29
11     46
12     23
13     63
14     47
15     24
16     35
17     21
18     27
19     45
20     55
21      9
22     29
23     21
24     48
25     50
26     22
27     22
28     41
29     29
       ..
388    21
389     6
390    23
391    51
392    13
393    47
394    29
395    18
396    24
397    48
398    22
399    31
400    30
401    38
402    22
403    17
404    43
405    20
406    23
407    50
408    29
409     3
410    29
411    37
412    28
413    29
414    39
415    38
416    29
417    29
Name: Age, Length: 418, dtype: int64

In [66]:
del train["Cabin"]
del test["Cabin"]
del train["Name"]
del test["Name"]
#del train["PassengerId"]

In [67]:
del test['Ticket']
del train['Ticket']

In [68]:
gene_type = train["Cabin"].iloc[0:]
gene_type[55]

KeyError: 'Cabin'

In [69]:


tt = [train, test]

for dataset in tt:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

train.head()

for dataset in tt:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype('int')

i=0
#for dataset in tt:
    #dataset['Cabin'] = dataset['Cabin'].map( {0:0,(gene_type[i]!=0):1} )
    #i+=1

In [70]:
train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,0,22.0,1,0,7.2500,0
1,2,1,1,1,38.0,1,0,71.2833,1
2,3,1,3,1,26.0,0,0,7.9250,0
3,4,1,1,1,35.0,1,0,53.1000,0
4,5,0,3,0,35.0,0,0,8.0500,0
5,6,0,3,0,29.0,0,0,8.4583,2
6,7,0,1,0,54.0,0,0,51.8625,0
7,8,0,3,0,2.0,3,1,21.0750,0
8,9,1,3,1,27.0,0,2,11.1333,0
9,10,1,2,1,14.0,1,0,30.0708,1


In [71]:
test

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,0,34.5,0,0,7.8292,2
1,893,3,1,47.0,1,0,7.0000,0
2,894,2,0,62.0,0,0,9.6875,2
3,895,3,0,27.0,0,0,8.6625,0
4,896,3,1,22.0,1,1,12.2875,0
5,897,3,0,14.0,0,0,9.2250,0
6,898,3,1,30.0,0,0,7.6292,2
7,899,2,0,26.0,1,1,29.0000,0
8,900,3,1,18.0,0,0,7.2292,1
9,901,3,0,21.0,2,0,24.1500,0


In [ ]:
#train["Cabin"] = train["Cabin"].replace(to_replace=[None],value=[0])
#test["Cabin"] = test["Cabin"].replace(to_replace=[None],value=[0])

In [72]:
test["Fare"] = test["Fare"].replace(to_replace=[None],value=[9])

In [73]:
import numpy as np
x=np.ones((9,891))
for i in range(0,9):
    x[i]=np.array(train.iloc[:,[i]]).T
y=np.array(x[1])
xp=np.array(x[0])
x=np.array(x[2:]).T
x=np.vstack((xp.T,x.T))
x=x.T
x.shape

(891, 8)

In [74]:
xt=np.ones((8,418))
for i in range(0,8):
    xt[i]=np.array(test.iloc[:,[i]]).T
xt=np.array(xt).T
xt.shape

(418, 8)

In [81]:
%%time
from sklearn import ensemble , cross_validation, metrics,tree,manifold,linear_model,model_selection, ensemble
import xgboost as xgb
feat=["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"]
estimator = xgb.XGBClassifier(learning_rate=0.01, max_depth=200, n_estimators=300, min_child_weight=4,feature_names=feat)
print(cross_validation.cross_val_score(estimator, x, y,scoring = 'roc_auc', cv = 10).mean())

0.868601349631
CPU times: user 2.36 s, sys: 39.9 ms, total: 2.4 s
Wall time: 2.41 s


In [ ]:
%%time

tree_clf=ensemble.RandomForestClassifier(n_estimators=200,max_depth=200)
print(cross_validation.cross_val_score(tree_clf, x, y,scoring = 'roc_auc', cv = 10).mean())

In [ ]:
tree_clf.fit(x,y)
tree_clf.predict(xt)

In [82]:
estimator.fit(x,y)
y_pred = estimator.predict(xt)
l=xt.T
m=l[0]
sub=np.vstack((m,y_pred)).T
np.round(sub[1])

array([ 893.,    0.])

In [83]:
subpd=pd.DataFrame(sub)
subpd=subpd.rename(columns = {0:'PassengerId',1:'Survived'})
subpd=subpd.astype("int")
subpd

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


In [84]:
subpd.to_csv('out.csv',sep=",",index=False)